In [ ]:
!pip install gradio pandas groq sentence-transformers chromadb langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 61.1 MB/s eta 0

In [ ]:
import gradio as gr
import datetime
import pandas as pd
from groq import Groq
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import hashlib
from typing import TypedDict, Optional, List
from langgraph.graph import StateGraph, END
import json
import tempfile
import subprocess
import os

In [ ]:
from google.colab import userdata
api_key_coder =userdata.get('coder')

In [ ]:
# ---------------------------
# 1. Define State
# ---------------------------
class CodeAssistantState(TypedDict):
    user_input: str
    similar_examples: Optional[List[str]]
    generated_code: Optional[str]
    error: Optional[str]
    task_type: Optional[str]  # "generate" or "explain"
    evaluation_result: Optional[str]

In [ ]:
# ---------------------------
# 2. Initialize Components
# ---------------------------
# Load data
df = pd.read_parquet("hf://datasets/openai/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet")
extracted_data = df[['task_id', 'prompt', 'canonical_solution']]

# Initialize models and DB
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
groq_client = Groq(api_key=api_key_coder)  # استبدل بمفتاح API الفعلي

client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    persist_directory="rag_db"
))
collection = client.get_or_create_collection(
    name="code_examples",
    metadata={"hnsw:space": "cosine"}
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

# ---------------------------
# 3. Define Nodes
# ---------------------------

def initialize_db(state: CodeAssistantState):
    try:
        for _, row in extracted_data.iterrows():
            embedding = embedding_model.encode([row['prompt'].strip()])[0]
            doc_id = hashlib.md5(row['prompt'].encode()).hexdigest()
            collection.add(
                documents=[row['canonical_solution'].strip()],
                metadatas=[{"prompt": row['prompt'], "type": "code_example"}],
                ids=[doc_id],
                embeddings=[embedding]
            )
        return state
    except Exception as e:
        state["error"] = f"DB initialization failed: {str(e)}"
        return state


In [ ]:
def retrieve_examples(state: CodeAssistantState):
    try:
        embedding = embedding_model.encode([state["user_input"]])[0]
        results = collection.query(
            query_embeddings=[embedding],
            n_results=2
        )
        state["similar_examples"] = results['documents'][0] if results['documents'] else None
        return state
    except Exception as e:
        state["error"] = f"Retrieval failed: {str(e)}"
        return state


In [ ]:
def classify_task_llm(state: CodeAssistantState) -> CodeAssistantState:
    if not isinstance(state, dict):
        raise ValueError("State must be a dictionary")

    if "user_input" not in state or not state["user_input"].strip():
        state["error"] = "No user input provided for classification"
        state["task_type"] = "generate"  # Default to code generation
        return state

    try:
        prompt = f"""You are a helpful code assistant. Classify the user request as one of the following tasks:
- "generate": if the user wants to write or generate code
- "explain": if the user wants to understand what a code snippet does
- "test": if the user wants to test existing code

Return ONLY a JSON object in the format: {{"task": "...", "user_input": "..."}} — no explanation.

User request: {state["user_input"]}
"""
        completion = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "Classify code-related user input. Respond with ONLY JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=200,
            response_format={"type": "json_object"}
        )

        content = completion.choices[0].message.content.strip()

        try:
            result = json.loads(content)
            if not isinstance(result, dict):
                raise ValueError("Response is not a JSON object")
        except (json.JSONDecodeError, ValueError) as e:
            state["error"] = f"Invalid response format from LLM: {str(e)}. Content: {content}"
            state["task_type"] = "generate"  # Fallback to code generation
            return state

        task_type = result.get("task", "").lower()
        if task_type not in ["generate", "explain", "test"]:
            state["error"] = f"Invalid task type received: {task_type}"
            task_type = "generate"  # Default to generation

        state["task_type"] = task_type
        state["user_input"] = result.get("user_input", state["user_input"])
        return state

    except Exception as e:
        state["error"] = f"LLM-based classification failed: {str(e)}"
        state["task_type"] = "generate"  # Fallback to code generation
        return state

In [ ]:
def test_code(state: CodeAssistantState) -> CodeAssistantState:
    if not isinstance(state, dict):
        raise ValueError("State must be a dictionary")

    if "user_input" not in state or not state["user_input"].strip():
        state["error"] = "Please provide the code you want to test"
        return state

    try:
        messages = [
            {"role": "system", "content": """You are a Python testing expert. Generate unit tests for the provided code.
Return the test code in the following format:
```python
# Test code here
```"""},
            {"role": "user", "content": f"Generate comprehensive unit tests for this Python code:\n\n{state['user_input']}"}
        ]

        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.5,
            max_tokens=2048,
        )

        test_code = completion.choices[0].message.content
        if test_code.startswith('```python'):
            test_code = test_code[9:-3] if test_code.endswith('```') else test_code[9:]
        elif test_code.startswith('```'):
            test_code = test_code[3:-3] if test_code.endswith('```') else test_code[3:]

        state["generated_tests"] = test_code.strip()
        state["metadata"] = {
            "model": "llama-3.3-70b-versatile",
            "timestamp": datetime.datetime.now().isoformat()
        }

        # Execute the tests and capture results
        try:
            # Create a temporary file to store the original code
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as code_file:
                code_file.write(state['user_input'])
                code_file_path = code_file.name

            # Create a temporary file to store the test code
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as test_file:
                test_file.write(test_code)
                test_file_path = test_file.name

            # Run the tests and capture output
            result = subprocess.run(
                ['python', test_file_path],
                capture_output=True,
                text=True,
                timeout=10
            )

            state["test_results"] = {
                "returncode": result.returncode,
                "stdout": result.stdout,
                "stderr": result.stderr
            }

            # Clean up temporary files
            os.unlink(code_file_path)
            os.unlink(test_file_path)

        except Exception as e:
            state["test_error"] = f"Error executing tests: {str(e)}"

        print(f"\nGenerated Tests:\n{test_code.strip()}\n")
        if "test_results" in state:
            print(f"Test Execution Results:\n{state['test_results']['stdout']}")
            if state["test_results"]["stderr"]:
                print(f"Errors:\n{state['test_results']['stderr']}")

        return state

    except Exception as e:
        state["error"] = f"Error generating tests: {str(e)}"
        return state

In [ ]:

def generate_code(state: CodeAssistantState) -> CodeAssistantState:
    if not isinstance(state, dict):
        raise ValueError("State must be a dictionary")

    if "user_input" not in state or not state["user_input"].strip():
        state["error"] = "Please enter your code request"
        return state

    try:
        messages = [
            {"role": "system", "content": "You are a Python coding assistant. Return only clean, production-ready code."},
            {"role": "user", "content": state["user_input"].strip()}
        ]

        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=2048,
        )

        code = completion.choices[0].message.content
        if code.startswith('```python'):
            code = code[9:-3] if code.endswith('```') else code[9:]
        elif code.startswith('```'):
            code = code[3:-3] if code.endswith('```') else code[3:]

        state["generated_code"] = code.strip()
        state["metadata"] = {
            "model": "llama-3.3-70b-versatile",
            "timestamp": datetime.datetime.now().isoformat()
        }

        # سطر طباعة النتيجة المضافة
        print(f"\nGenerated Code:\n{code.strip()}\n")

        return state

    except Exception as e:
        state["error"] = f"Error generating code: {str(e)}"
        return state


In [ ]:
def explain_code(state: CodeAssistantState) -> CodeAssistantState:
    try:
        messages = [
            {"role": "system", "content": "You are a Python expert. Explain what the following code does in plain language."},
            {"role": "user", "content": state["user_input"].strip()}
        ]

        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.5,
            max_tokens=1024
        )

        explanation = completion.choices[0].message.content.strip()
        state["generated_code"] = explanation
        state["metadata"] = {
            "model": "llama-3.3-70b-versatile",
            "timestamp": datetime.datetime.now().isoformat()
        }

        # سطر طباعة النتيجة المضافة
        print(f"Explanation:\n{explanation}")

        return state

    except Exception as e:
        state["error"] = f"Error explaining code: {str(e)}"
        return state

In [ ]:
# ---------------------------
# 4. Build StateGraph Workflow (محدث)
# ---------------------------
workflow = StateGraph(CodeAssistantState)

# إضافة جميع العقد بما فيها العقدة الجديدة
workflow.add_node("initialize_db", initialize_db)
workflow.add_node("retrieve_examples", retrieve_examples)
workflow.add_node("classify_task", classify_task_llm)
workflow.add_node("generate_code", generate_code)
workflow.add_node("explain_code", explain_code)
workflow.add_node("test_code", test_code)  # العقدة الجديدة

# تحديد نقطة البداية والروابط الأساسية
workflow.set_entry_point("initialize_db")
workflow.add_edge("initialize_db", "retrieve_examples")
workflow.add_edge("retrieve_examples", "classify_task")

# تحديث الروابط الشرطية لتشمل خيار الاختبار
workflow.add_conditional_edges(
    "classify_task",
    lambda state: state["task_type"],
    {
        "generate": "generate_code",
        "explain": "explain_code",
        "test": "test_code"  # الرابط الجديد
    }
)

# إضافة روابط النهاية لجميع العقد
workflow.add_edge("generate_code", END)
workflow.add_edge("explain_code", END)
workflow.add_edge("test_code", END)  # الرابط الجديد

# تجميع التدفق النهائي
app_workflow = workflow.compile()

# ---------------------------
# 5. Create Gradio Interface
# ---------------------------
def process_input(user_input: str):
    """Function that will be called by Gradio to process user input"""
    initial_state = {
        "user_input": user_input,
        "similar_examples": None,
        "generated_code": None,
        "error": None,
        "task_type": None
    }

    result = app_workflow.invoke(initial_state)

    if result.get("error"):
        return f"Error: {result['error']}"

    if result["task_type"] == "generate":
        return f"Generated Code:\n\n{result['generated_code']}"
    else:
        return f"Code Explanation:\n\n{result['generated_code']}"

# تعريف واجهة Gradio
# Define Gradio interface
with gr.Blocks(title="Smart Code Assistant") as demo:
    gr.Markdown("""
    # Smart Code Assistant
    Enter your request either to generate new code or to explain existing code
    """)

    with gr.Row():
        input_text = gr.Textbox(label="Enter your request", placeholder="Example: Write a function to add two numbers... or Explain this code...")
        output_text = gr.Textbox(label="Result", interactive=False)

    submit_btn = gr.Button("Execute")
    submit_btn.click(fn=process_input, inputs=input_text, outputs=output_text)

    # Quick examples
    gr.Examples(
    examples=[
        ["Write a Python function to add two numbers"],
        ["Explain this code: for i in range(5): print(i)"],
        ["Create a function to convert temperature from Fahrenheit to Celsius"],
        ["test for i in range(3): print('Hello from test', i)"]
       ],

        inputs=input_text
    )

# تشغيل الواجهة
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab9cdf65b408adc24d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!wget https://raw.githubusercontent.com/google-research/google-research/master/mbpp/mbpp.jsonl


#get data from mbpp.jsonl
with open("mbpp.jsonl", "r") as f:
    mbpp_data = [json.loads(line) for line in f]
test_data = mbpp_data[:10]

--2025-07-31 20:07:05--  https://raw.githubusercontent.com/google-research/google-research/master/mbpp/mbpp.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 563743 (551K) [text/plain]
Saving to: ‘mbpp.jsonl’

mbpp.jsonl          100%[===================>] 550.53K  --.-KB/s    in 0.05s   

2025-07-31 20:07:05 (10.1 MB/s) - ‘mbpp.jsonl’ saved [563743/563743]



In [ ]:
import pandas as pd

df = pd.DataFrame(test_data)
df.head()


,text,code,task_id,test_setup_code,test_list,challenge_test_list
0,Write a function to find the minimum cost path...,"R = 3\r\nC = 3\r\ndef min_cost(cost, m, n): \r...",1,,"[assert min_cost([[1, 2, 3], [4, 8, 2], [1, 5,...",[]
1,Write a function to find the similar elements ...,"def similar_elements(test_tup1, test_tup2):\r\...",2,,"[assert similar_elements((3, 4, 5, 6),(5, 7, 4...",[]
2,Write a python function to identify non-prime ...,import math\r\ndef is_not_prime(n):\r\n res...,3,,"[assert is_not_prime(2) == False, assert is_no...",[]
3,Write a function to find the largest integers ...,import heapq as hq\r\ndef heap_queue_largest(n...,4,,"[assert heap_queue_largest( [25, 35, 22, 85, 1...",[]
4,Write a function to find the number of ways to...,def count_ways(n): \r\n\tA = [0] * (n + 1) \r\...,5,,"[assert count_ways(2) == 3, assert count_ways(...",[]


In [ ]:
extracted_test_data = df[['task_id', 'text', 'code']]

In [ ]:
extracted_test_data.head()

,task_id,text,code
0,1,Write a function to find the minimum cost path...,"R = 3\r\nC = 3\r\ndef min_cost(cost, m, n): \r..."
1,2,Write a function to find the similar elements ...,"def similar_elements(test_tup1, test_tup2):\r\..."
2,3,Write a python function to identify non-prime ...,import math\r\ndef is_not_prime(n):\r\n res...
3,4,Write a function to find the largest integers ...,import heapq as hq\r\ndef heap_queue_largest(n...
4,5,Write a function to find the number of ways to...,def count_ways(n): \r\n\tA = [0] * (n + 1) \r\...


In [ ]:
# Initialize client with error handling
try:
    groq_client = Groq(api_key=api_key_coder)  # Use environment variable
except Exception as e:
    print(f"Error initializing client: {e}")
    exit()

def evaluate_with_llama(df):
    results = []
    for index, row in df.iterrows():
        try:
            # Generate code
            initial_state = {"user_input": row['text']}
            result = app_workflow.invoke(initial_state)
            generated_code = result.get("generated_code", "")

            # Prepare request for code comparison
            comparison_response = groq_client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {
                        "role": "system",
                        "content": "Compare the two codes precisely. Respond with only 'Pass' or 'Fail'."
                    },
                    {
                        "role": "user",
                        "content": f"Task: {row['text']}\n\nCode 1:\n{generated_code}\n\nCode 2:\n{row['code']}"
                    }
                ],
                temperature=0.1
            )
            comparison_verdict = comparison_response.choices[0].message.content

            # New: Check if generated code is executable
            execution_response = groq_client.chat.completions.create(
                model="llama3-70b-8192",  # يمكن استخدام نموذج مختلف هنا إذا أردت
                messages=[
                    {
                        "role": "system",
                        "content": "Analyze if this code is syntactically correct and executable. Respond with only 'Executable' or 'Not Executable'."
                    },
                    {
                        "role": "user",
                        "content": f"Code to analyze:\n{generated_code}"
                    }
                ],
                temperature=0.1
            )
            execution_verdict = execution_response.choices[0].message.content

            results.append({
                "task_id": row['task_id'],
                "comparison_result": "✅ Pass" if "Pass" in comparison_verdict else "❌ Fail",
                "execution_result": "✅ Executable" if "Executable" in execution_verdict else "❌ Not Executable",

            })

        except Exception as e:
            print(f"Error processing task {row['task_id']}: {e}")
            results.append({
                "task_id": row['task_id'],
                "comparison_result": f"Error: {str(e)}",
                "execution_result": f"Error: {str(e)}",
                "generated_code": ""
            })

    return pd.DataFrame(results)

In [ ]:
results=evaluate_with_llama(extracted_test_data)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Generated Code:
**Minimum Cost Path Function**

### Problem Description

This function finds the minimum cost path to reach a given position `(m, n)` from the origin `(0, 0)` in a 2D cost matrix `cost[][]`.

### Code

```python
def min_cost_path(cost):
    """
    Find the minimum cost path to reach (m, n) from (0, 0) in a given cost matrix.

    Args:
        cost (list[list[int]]): A 2D list representing the cost matrix.

    Returns:
        int: The minimum cost to reach the destination.
    """
    rows, cols = len(cost), len(cost[0])
    dp = [[0] * cols for _ in range(rows)]

    # Initialize the first cell
    dp[0][0] = cost[0][0]

    # Fill the first row
    for col in range(1, cols):
        dp[0][col] = dp[0][col - 1] + cost[0][col]

    # Fill the first column
    for row in range(1, rows):
        dp[row][0] = dp[row - 1][0] + cost[row][0]

    # Fill the rest of the table
    for row in range(1, rows):
        for col in range(1, cols):
            dp[row][col] = cost[

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Generated Code:
### Finding Similar Elements in Tuple Lists

Here's a Python function that finds similar elements from two given tuple lists:

```python
def find_similar_elements(tuple_list1, tuple_list2):
    """
    This function finds similar elements from two given tuple lists.

    Args:
        tuple_list1 (list): The first list of tuples.
        tuple_list2 (list): The second list of tuples.

    Returns:
        list: A list of tuples that are common to both input lists.
    """
    # Convert the lists to sets for efficient lookups
    set1 = set(tuple_list1)
    set2 = set(tuple_list2)

    # Find the intersection of the two sets
    similar_elements = set1.intersection(set2)

    # Convert the set back to a list and return it
    return list(similar_elements)


# Example usage:
tuple_list1 = [(1, 2), (3, 4), (5, 6)]
tuple_list2 = [(3, 4), (5, 6), (7, 8)]

similar = find_similar_elements(tuple_list1, tuple_list2)
print("Similar elements:", similar)
```

**Output:**
```
Simil

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


Generated Code:
def square_elements(input_list):
    """
    This function takes a list of numbers and returns a new list 
    containing the squares of each element in the input list.

    Args:
        input_list (list): A list of numbers.

    Returns:
        list: A list of squares of the input elements.
    """
    # Use map function with lambda function to square each element
    squared_list = list(map(lambda x: x ** 2, input_list))
    return squared_list

# Example usage:
if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    squared_numbers = square_elements(numbers)
    print("Input List:", numbers)
    print("Squared List:", squared_numbers)
```

**Explanation:**

*   We define a function `square_elements` that takes a list as an input.
*   Inside the function, we use the built-in `map` function in combination with a lambda function to square each element in the input list.
*   The lambda function `lambda x: x ** 2` takes an element `x` and returns its square.
*   T

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Generated Code:
### Minimum Rotations Function
```python
def min_rotations(s: str) -> int:
    """
    Find the minimum number of rotations required to get the same string.

    Args:
    s (str): The input string.

    Returns:
    int: The minimum number of rotations required.
    """
    if not s:
        return 0

    # Concatenate the string with itself
    double_s = s + s

    # Find the first occurrence of the original string in the concatenated string
    for i in range(len(s)):
        if double_s[i:i+len(s)] == s:
            return i

    return len(s)

# Example usage
if __name__ == "__main__":
    print(min_rotations("abcabc"))  # Output: 0
    print(min_rotations("abcdabcd"))  # Output: 0
    print(min_rotations("abababab"))  # Output: 0
    print(min_rotations("abcde"))  # Output: 5
```

In this code:

1. We define a function `min_rotations` that takes a string `s` as input and returns the minimum number of rotations required to get the same string.
2. We concatenate t

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Generated Code:
### Get N Smallest Items Function
#### Overview
The following Python function uses the built-in `heapq` module to efficiently retrieve the n smallest items from a dataset.

#### Code
```python
import heapq

def get_n_smallest(dataset, n):
    """
    Retrieves the n smallest items from a dataset.

    Args:
    - dataset (list): The input dataset.
    - n (int): The number of smallest items to retrieve.

    Returns:
    - list: A list of the n smallest items in the dataset.
    """
    if n > len(dataset):
        raise ValueError("n cannot be greater than the dataset size")

    # Use heapq.nsmallest to get the n smallest items
    smallest_items = heapq.nsmallest(n, dataset)

    return smallest_items

# Example usage
if __name__ == "__main__":
    dataset = [12, 45, 7, 23, 56, 89, 34]
    n = 3
    smallest = get_n_smallest(dataset, n)
    print(f"The {n} smallest items in the dataset are: {smallest}")
```

#### Explanation
This function takes a dataset and the num

In [ ]:
display(results)

,task_id,comparison_result,execution_result
0,1,✅ Pass,✅ Executable
1,2,❌ Fail,✅ Executable
2,3,❌ Fail,✅ Executable
3,4,✅ Pass,✅ Executable
4,5,✅ Pass,✅ Executable
5,6,✅ Pass,✅ Executable
6,7,✅ Pass,✅ Executable
7,8,✅ Pass,✅ Executable
8,9,✅ Pass,✅ Executable
9,10,✅ Pass,✅ Executable
